In [35]:
import numpy as np
import pandas as pd
import csv
import datetime
from datapathfile import path_lookup_dir, datafile

In [36]:
def listcolumn(x):
    x = x.translate(None, '"')
    x = x.translate(None, ' ')
    x = x[1:-1].split(",")
    j = x[-1]
    return j

In [37]:
def TrueFalse(value):
    if value[0]=='F':
        value = False
    else:
        value = True
    return value

In [38]:
filename='ATA2_data.csv'

In [39]:
reader = list(csv.reader(open(path_lookup_dir+datafile, 'rb'), delimiter='\t'))
colheaders = reader[0] 
df = pd.DataFrame(reader[1:])
dffilter = df.iloc[:, 0:20]
dffilter.columns = [colheaders]

In [40]:
dffilter['installed'].unique()

array(['False', 'False2015-05-16 00:01:18', 'False2015-05-17 06:47:28',
       ..., 'False2015-07-24 16:45:05', 'False2015-07-24 16:51:24',
       'False2015-07-24 17:20:35'], dtype=object)

In [41]:
df.shape

(1645578, 39)

In [42]:
df = dffilter

In [43]:
df['impression'] = df['impression'].apply(TrueFalse)

In [44]:
df['installed'] = df['installed'].apply(TrueFalse)

In [45]:
df['clicked'] = df['clicked'].apply(TrueFalse)

In [46]:
df.ix[:, 'datetime'] = pd.to_datetime(df['datetime'], format="%Y-%m-%d %H:%M:%S")
df.ix[:, 'app_type'] = df['app_categories'].apply(listcolumn)


In [47]:
df.shape

(1645578, 21)

In [48]:
drop_index=df[np.logical_and(
        np.logical_and(df['impression'], df['installed']),
        ~ df['clicked'])].index

In [49]:
empty_index = np.where(df['app_type'] == '')
ul_index = np.where(df['app_type'] == 'ul')
df = df.drop(df.index[empty_index])
df = df.drop(df.index[ul_index])
df = df.drop(df.index[drop_index])

In [50]:
df['weekday'] = df['datetime'].map(lambda x: x.isoweekday())
df['hours'] = df['datetime'].map(lambda x: x.hour)
df.ix[:, 'week'] = df['datetime'].map(lambda x:x.isocalendar()[1])

In [51]:
impressiondf = df.groupby('city')['impression','clicked'].sum()
impression_index = impressiondf[impressiondf['impression'] > 5].index

In [52]:
df=df[df.city.isin(impression_index)]

In [53]:
df.to_csv(filename)